In [5]:
from dotenv import load_dotenv
import os

load_dotenv()

token = os.environ.get('token')
GigaChatKey = os.environ.get('GigaChatKey')

In [3]:
# Requirements
!pip install aiogram aiosqlite apscheduler
!pip install nest_asyncio
!pip install langchain-gigachat langchain-community
import nest_asyncio
nest_asyncio.apply()

In [6]:
#Пример формирования диалога с GigaChat
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_gigachat.chat_models import GigaChat

llm = GigaChat(
    credentials=GigaChatKey,
    scope="GIGACHAT_API_PERS",
    model="GigaChat",
    verify_ssl_certs=False, # Отключает проверку наличия сертификатов НУЦ Минцифры
    streaming=False,
)


system_messages = [
    SystemMessage(
        content="Ты профессианальный тренер покемонов, который отлично разрирается в навыках любого покемона. Твоя задача - помагать со всеми приходящими вопросами от тренеров-новичков."
    )
]

user_messages = {}

pokemon_predict_template = """Я отправлю тебе моих покемонов, покемонов соперника и их уровни. Опредили тип этих покемонов исходя из их названия, и твоих знаний о них. Если лучшего варианта нет, скажи об этом, и очень кратко приведи типы покемонов, которые мне нужны, но все равно ты обязан предложить хотя бы какую-нибудь комбинацию именно моих покемонов, которыми я сейчас обладаю, и не предлагать покемонов, которых у меня нет.
Все мои покемоны: {}. Далеее идут покемоны противника
Покемоны противника: {}.
Твоя задача - предложить лучший вариант комбинации моих покемонов, которые буду сражаться против покемонов противника. Если покемон написан несколько раз - значит у меня их несколько. 
Также ты обязан задать мне ровно один дополнительный вопрос, которые поможет тебе улучшить ответ.
"""

async def start_pokemon_prediction(user_id, team_pokemons, enemy_pokemons):
    print(team_pokemons)
    team_pokemons_str = ", ".join(map(lambda x: f"{x[2]}: lvl{x[3]}", team_pokemons))
    enemy_pokemons_str = ", ".join(map(lambda x: f"{x[0]}: lvl{x[1]}", enemy_pokemons))

    msg = pokemon_predict_template.format(team_pokemons_str, enemy_pokemons_str)
    llm_messages = system_messages + user_messages.get(user_id, [])
    llm_messages.append(HumanMessage(msg))

    print(llm_messages)

    response = await llm.ainvoke(llm_messages)
    llm_messages.append(response)
    user_messages[user_id] = llm_messages

    print(response)
    return response


async def end_pokemon_prediction(user_id, text):
    msg = user_messages.get(user_id, None)

    if not msg:
        return "Something went wrong"
    
    del user_messages[user_id]

    msg.append(HumanMessage(text))
    response = await llm.ainvoke(msg)

    return response



In [11]:
from aiogram import Bot, Dispatcher
from aiogram.client.default import DefaultBotProperties
from aiogram.enums import ParseMode
from aiogram.fsm.storage.memory import MemoryStorage
from aiogram import Router, F
from aiogram.filters import CommandStart, Command, CommandObject
from aiogram.types import Message
from aiogram.types import KeyboardButton, ReplyKeyboardMarkup, KeyboardButtonPollType
from aiogram.utils.keyboard import ReplyKeyboardBuilder
from aiogram.types import InlineKeyboardMarkup, InlineKeyboardButton, WebAppInfo
from aiogram.utils.keyboard import InlineKeyboardBuilder
from aiogram.types import BotCommand, BotCommandScopeDefault
from aiogram.types import CallbackQuery
from aiogram.utils.chat_action import ChatActionSender
from aiogram.types import ReplyKeyboardRemove
from aiogram.fsm.context import FSMContext
from aiogram.fsm.state import State, StatesGroup
from typing import Any, Awaitable, Callable, Dict
from aiogram import BaseMiddleware
from aiogram.types import TelegramObject
from html import escape

import datetime
import asyncio
import logging
import aiosqlite

from apscheduler.schedulers.asyncio import AsyncIOScheduler


class RegCheckMiddleware(BaseMiddleware):
    async def __call__(
        self,
        handler: Callable[[TelegramObject, Dict[str, Any]], Awaitable[Any]],
        event: TelegramObject,
        data: Dict[str, Any]
    ) -> Any:

        print(f"CHECK! {event.from_user.id}")
        if not await dbcontroller.get_user(event.from_user.id):
            print("UNREGISTERED USER!")
            await event.answer("Для начала используйте /start, чтобы зарегестрироватся.")
            return

        result = await handler(event, data)

        return result
    

class LoggerMiddleware(BaseMiddleware):
    async def __call__(self, handler, event: Message, data: dict):
        user_id = event.from_user.id
        username = event.from_user.username or "Unknown"
        action = event.text
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        with open("logs.csv", "a") as log_file:
            log_file.write(f"{timestamp},{user_id},{username},{action}\n")

        return await handler(event, data)


class Form(StatesGroup):
    nickname = State()
    team = State()

class FightStates(StatesGroup):
    enemy_pokemons = State()

class ClearStates(StatesGroup):
    clear_my_team = State()
    delete_pokemon = State()

class LLMStates(StatesGroup):
    strategy_question = State()


logging.basicConfig(force=True, level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

bot = Bot(token=token, default=DefaultBotProperties(parse_mode=ParseMode.HTML))
dp = Dispatcher(storage=MemoryStorage())

registered_router = Router(name="registered")
start_router = Router(name=__name__)


class DBController:
    @staticmethod
    async def add_user(id, name):
        async with aiosqlite.connect("users.db") as db:
            await db.execute("INSERT INTO users VALUES (?, ?)", (id, name))
            await db.commit()
    @staticmethod
    async def get_user(id):
        async with aiosqlite.connect("users.db") as db:
            async with db.execute("SELECT * FROM users WHERE id = ?", (id,)) as cursor:
                return await cursor.fetchone()
    @staticmethod
    async def get_user_by_nickname(name: str):
        async with aiosqlite.connect("users.db") as db:
            async with db.execute("SELECT * FROM users WHERE nickname = ?", (name,)) as cursor:
              return await cursor.fetchone()
    @staticmethod
    async def del_user(id):
      async with aiosqlite.connect("users.db") as db:
        await db.execute("DELETE FROM users WHERE id = ?", (id,))
        await db.commit()
    @staticmethod
    async def show_db():
        async with aiosqlite.connect('users.db') as db:
            async with db.execute("SELECT * FROM users") as cursor:
                async for row in cursor:
                    print(row)
    @staticmethod
    async def is_real_pokemon(name):
        async with aiosqlite.connect("users.db") as db:
            async with db.execute("SELECT * FROM pokemons WHERE name = ?", (name,)) as cursor:
              return await cursor.fetchone() is not None
    @staticmethod
    async def get_all_pokemons():
        async with aiosqlite.connect("users.db") as db:
            async with db.execute("SELECT * FROM pokemons") as cursor:
              return await cursor.fetchall()

    @staticmethod
    async def get_user_pokemons(id):
        print(id)
        async with aiosqlite.connect("users.db") as db:
            async with db.execute("SELECT * FROM user_pokemons WHERE user_id=?", (id,)) as cursor:
              return await cursor.fetchall() or []
                
    @staticmethod
    async def add_pokemon(id, pokemon_name, pokemon_lvl):
        async with aiosqlite.connect("users.db") as db:
            await db.execute("INSERT INTO user_pokemons VALUES (NULL, ?, ?, ?)", (id, pokemon_name, pokemon_lvl))
            await db.commit()

    @staticmethod
    async def add_real_pokemon(pokemon_name):
        async with aiosqlite.connect("users.db") as db:
            await db.execute("INSERT INTO pokemons VALUES (?)", (pokemon_name,))
            await db.commit()

    @staticmethod
    async def delete_user_pokemons(user_id):
        async with aiosqlite.connect("users.db") as db:
            await db.execute("DELETE FROM user_pokemons WHERE user_id=?", (user_id,))
            await db.commit()

    @staticmethod
    async def delete_user_pokemon(user_id, pokemon_name, pokemon_lvl):
        async with aiosqlite.connect("users.db") as db:
            await db.execute("DELETE FROM user_pokemons WHERE pokemon_id = "
                             "(SELECT MIN(pokemon_id) FROM user_pokemons WHERE user_id = ? AND pokemon_name=? AND pokemon_lvl=?)", (user_id, pokemon_name, pokemon_lvl))
            await db.commit()



dbcontroller = DBController()


@start_router.message(CommandStart())
async def cmd_start(message: Message, state: FSMContext):
    usr = await dbcontroller.get_user(message.from_user.id)

    if not usr:
        await message.answer(f'Привет, {message.from_user.last_name} {message.from_user.first_name}! Введите свой новый ник')
        await state.set_state(Form.nickname)
        return

    await message.answer(f"Привет, {usr[1]}. Используйте /help для помощи, или /, чтобы увидеть доступные команды.")
    return


@start_router.message(Command("help"))
async def help(message: Message):
    is_reg = bool(await dbcontroller.get_user(message.from_user.id))

    if not is_reg:
        await message.answer(f"/start - регистрация")
        return
    
    await message.answer(f"/strategy - анализ команды тренера и противника с последующими предложениями")


@dp.message(Form.nickname)
async def add_name(message: Message, state: FSMContext):
    print(message.text)
    if await dbcontroller.get_user_by_nickname(message.text):
        await message.answer(f"Ник занят. Пожалуйста, выберите другой.")
        return
    
    await state.update_data(nickname=message.text)
    await state.set_state(None)

    await dbcontroller.add_user(message.from_user.id, message.text)

    await message.answer(f"Теперь ваш ник {message.text}")


@dp.message(Command("real_pokemons"))
async def real_pokemons(message: Message):
    pkms = await dbcontroller.get_all_pokemons()
    print(pkms)
    msg = "\n".join(map(lambda x: x[0], pkms))

    if not msg:
        await message.answer("EMPTY")
        return
    
    await message.answer(msg)

@dp.message(Command("add_real_pokemon"))
async def real_pokemons(message: Message):
    name = message.text.split()[1]

    await dbcontroller.add_real_pokemon(name)
    await message.answer(f"Покемон {name} добавлен")


@registered_router.message(Command("clear_my_team"))
async def clear_my_team(message: Message, state: FSMContext):
    await dbcontroller.delete_user_pokemons(message.from_user.id)
    await message.answer("Your team is cleared")


@registered_router.message(LLMStates.strategy_question)
async def additional_strategy__question(message: Message, state: FSMContext):
    response = await end_pokemon_prediction(message.from_user.id, message.text)
    await message.answer(response.content)
    await message.answer("Диалог с GigaChat окончен.")
    await state.clear()
    await state.set_state(None)

# @start_router.callback_query(F.data == 'team_end')
@registered_router.message(Form.team)
async def add_team(message: Message, state: FSMContext):
    if isinstance(message, CallbackQuery) or message.text.lower() == "конец":
        await message.answer(f"Добавление покемонов закончено", reply_markup=ReplyKeyboardRemove())
        await state.set_state(None)
        return
        
    if len(message.text.split()) != 2:
        await message.answer(f"Введите всех своих покемонов в формате: POKEMON_NAME POKEMON_LVL")
        return
        
    name, lvl = message.text.split()
    if not lvl.isdigit():
        await message.answer(f"Lvl должен быть числом")
        return

    if not (await dbcontroller.is_real_pokemon(name)):
        await message.answer(f"Это не настоящий покемон (или его нет в системе)!")
        return

    await dbcontroller.add_pokemon(message.from_user.id, name, lvl)

    builder = ReplyKeyboardBuilder()
    builder.button(text="Конец", callback_data="конец")

    await message.answer(f"В вашу команду успешно добавлен {name}: lvl{lvl}", reply_markup=builder.as_markup(one_time_keyboard=True))

@registered_router.message(Command("update_team"))
@registered_router.message(F.text == "Enter team")
async def enter_team(message: Message, state: FSMContext):
    await state.set_state(Form.team)
    await message.answer(f"Введите всех своих покемонов в формате: POKEMON_NAME POKEMON_LVL")

@registered_router.message(Command("myteam"))
async def my_team(message: Message):
    pkms = await dbcontroller.get_user_pokemons(message.from_user.id)

    print("ASD, ", pkms)
    msg = "\n".join(map(lambda x: f"{x[2]}: lvl {x[3]}", pkms))

    if not msg:
        await message.answer("Ваша команда пуста.")
        return
    
    await message.answer(msg)


@registered_router.message(Command("delete_pokemon"))
async def delete_pokemon(message: Message, state: FSMContext):
    await state.set_state(ClearStates.delete_pokemon)

    kb_but = []

    for p in await dbcontroller.get_user_pokemons(message.from_user.id):
        kb_but.append([InlineKeyboardButton(text=f"{p[2]}: lvl {p[3]}", callback_data=f"{p[2]} {p[3]}")])

    if not kb_but:
        await bot.send_message(message.from_user.id, "Ваша команда пуста.")
        await state.set_state(None)
        return False
    
    kb_but.append([InlineKeyboardButton(text="Конец", callback_data="end")])
    
    kb = InlineKeyboardMarkup(inline_keyboard=kb_but, one_time_keyboard=True)

    await bot.send_message(message.from_user.id, "Выберите покемона для удаления", reply_markup=kb)
    return True

@registered_router.callback_query(ClearStates.delete_pokemon)
async def delete_pokemon_query(callback_query: CallbackQuery, state: FSMContext):
    await bot.delete_message(callback_query.from_user.id, callback_query.message.message_id)
    if callback_query.data == "end":
        await callback_query.answer("Удаление покемонов закончено")
        await state.set_state(None)
        return

    pokemon_name, pokemon_level = callback_query.data.split()
    await dbcontroller.delete_user_pokemon(callback_query.from_user.id, pokemon_name, int(pokemon_level))

    await callback_query.message.answer(f"Покемон {pokemon_name}: lvl {pokemon_level} удален из вашей команды")

    if not await delete_pokemon(callback_query, state=state):
        await bot.delete_message(callback_query.from_user.id, callback_query.message.message_id)



@registered_router.message(Command("strategy"))
async def fight(message: Message, state: FSMContext):
    team_pokemons = await dbcontroller.get_user_pokemons(message.from_user.id)
    if not team_pokemons:
        await message.answer("Похоже, ваша команда пуста..")
        return
    
    await state.update_data(enemy_pokemons=[])
    await state.set_state(FightStates.enemy_pokemons)
    await message.answer("Введите покемонов из команды противника")

async def start_fight_prediction(message: Message, state: FSMContext, enemy_pokemons):
    team_pokemons = await dbcontroller.get_user_pokemons(message.from_user.id)
    if not team_pokemons:
        await message.answer("Похоже, ваша команда пуста..")
        return

    prediction = await start_pokemon_prediction(message.from_user.id, team_pokemons, enemy_pokemons)
    await state.set_state(LLMStates.strategy_question)

    await message.answer(prediction.content)

@registered_router.message(FightStates.enemy_pokemons)
async def fight_pokemons(message: Message, state: FSMContext):
    if isinstance(message, CallbackQuery) or message.text.lower() == "конец":
        await message.answer(f"Пожалуйста, подождите ответа от GigaChat.", reply_markup=ReplyKeyboardRemove())
        await state.set_state(None)
        await start_fight_prediction(message, state, enemy_pokemons=(await state.get_data())["enemy_pokemons"])
        return
        
    if len(message.text.split()) != 2:
        await message.answer(f"Введите в формате: POKEMON_NAME POKEMON_LVL")
        return
        
    name, lvl = message.text.split()
    if not lvl.isdigit():
        await message.answer(f"Lvl должен быть числом")
        return

    if not (await dbcontroller.is_real_pokemon(name)):
        await message.answer(f"Это не настоящий покемон (или его нет в системе)!")
        return

    enemy_pokemons = (await state.get_data())["enemy_pokemons"]
    enemy_pokemons.append((name, lvl))
    await state.update_data(enemy_pokemons=enemy_pokemons)

    builder = ReplyKeyboardBuilder()
    builder.button(text="Конец", callback_data="конец") 

    await message.answer(f"Добавлен покемон противника {name}: lvl{lvl}", reply_markup=builder.as_markup(one_time_keyboard=True))


@registered_router.message(F.text)
async def cmd_mes(message: Message):
    await message.answer(f"Это понимать я еще не научился")

async def send_msg(dp):
  async with aiosqlite.connect('users.db') as db:
    async with db.execute("SELECT * FROM users") as cursor:
      async for row in cursor:
        await bot.send_message(chat_id=row[0], text='Не забывайте обновлять информацию о своей команде!')

async def start_bot():
    commands = [BotCommand(command='start', description='Старт'),
                BotCommand(command='update_team', description='Заполнить команду покемонов'),
                BotCommand(command='myteam', description='Посмотреть моих покемонов'),
                BotCommand(command='clear_my_team', description='Очистить команду'),
                BotCommand(command='delete_pokemon', description='Удалить покемонов'),
                BotCommand(command='strategy', description='Анализ покемонов для составления стратегии'),
                BotCommand(command='real_pokemons', description='Список сущ. в сист. покемонов'),
                BotCommand(command='add_real_pokemon', description='Добавление в сист. покемонов'),
    ]
    await bot.set_my_commands(commands, BotCommandScopeDefault())

async def start_db():
    # Подключение к базе данных (если файл не существует, он будет создан)

    async with aiosqlite.connect('users.db') as db:
        await db.execute('DROP TABLE IF EXISTS users')
        await db.execute('DROP TABLE IF EXISTS user_pokemons')
        # Создание таблицы, если она не существует
        await db.execute('''
            CREATE TABLE IF NOT EXISTS users (
                id INTEGER PRIMARY KEY,
                nickname TEXT
            )
        ''')
        await db.execute('''
            CREATE TABLE IF NOT EXISTS pokemons (
                name TEXT PRIMARY KEY
            )
        ''')

        await db.execute('''
            CREATE TABLE IF NOT EXISTS user_pokemons (
                pokemon_id INTEGER PRIMARY KEY AUTOINCREMENT,
                user_id INT,
                pokemon_name TEXT,
                pokemon_lvl INT,
                FOREIGN KEY (user_id) REFERENCES users (id) ON DELETE CASCADE
            )
        ''')
        # Сохранение изменений
        await db.commit()

async def main(): #Основная асинхронная функция, которая будет запускаться при старте бота.
    scheduler = AsyncIOScheduler(timezone='Europe/Moscow')
    job = scheduler.add_job(send_msg, 'interval', seconds=500, args=(dp,))
    scheduler.start()
    
    registered_router.message.outer_middleware(RegCheckMiddleware())
    registered_router.message.middleware(LoggerMiddleware())
    start_router.message.middleware(LoggerMiddleware())
    
    dp.include_router(start_router)
    dp.include_router(registered_router) 
    dp.startup.register(start_bot)
    dp.startup.register(start_db)
    
    try:
      print("Бот запущен...")
      await bot.delete_webhook(drop_pending_updates=True)
      await dp.start_polling(bot, allowed_updates=dp.resolve_used_update_types()) # Запускаем бота в режиме опроса (polling). Бот начинает непрерывно запрашивать обновления с сервера Telegram и обрабатывать их
    finally:
      scheduler.remove_job(job.id)
      await bot.session.close()
      print("Бот остановлен")

asyncio.run(main())

2025-01-19 16:58:53,726 - apscheduler.scheduler - INFO - Adding job tentatively -- it will be properly scheduled when the scheduler starts
2025-01-19 16:58:53,727 - apscheduler.scheduler - INFO - Added job "send_msg" to job store "default"
2025-01-19 16:58:53,727 - apscheduler.scheduler - INFO - Scheduler started


Бот запущен...


2025-01-19 16:58:54,040 - aiogram.dispatcher - INFO - Start polling
2025-01-19 16:58:54,098 - aiogram.dispatcher - INFO - Run polling for bot @TestOGE8_bot id=5141660175 - 'PokemonStrategy'


CHECK! 887383975
UNREGISTERED USER!


2025-01-19 16:58:57,082 - aiogram.event - INFO - Update id=708829274 is handled. Duration 329 ms by bot id=5141660175
2025-01-19 17:07:13,729 - apscheduler.executors.default - INFO - Running job "send_msg (trigger: interval[0:08:20], next run at: 2025-01-19 17:15:33 MSK)" (scheduled at 2025-01-19 17:07:13.726491+03:00)
2025-01-19 17:07:13,743 - apscheduler.executors.default - INFO - Job "send_msg (trigger: interval[0:08:20], next run at: 2025-01-19 17:15:33 MSK)" executed successfully
2025-01-19 17:20:00,245 - aiogram.dispatcher - ERROR - Failed to fetch updates - TelegramNetworkError: HTTP Client says - Request timeout error
2025-01-19 17:20:00,246 - aiogram.dispatcher - WARNING - Sleep for 1.000000 seconds and try again... (tryings = 0, bot id = 5141660175)
2025-01-19 17:20:11,572 - aiogram.dispatcher - INFO - Connection established (tryings = 1, bot id = 5141660175)
2025-01-19 17:26:28,347 - aiogram.dispatcher - ERROR - Failed to fetch updates - TelegramNetworkError: HTTP Client say

nikita0607


2025-01-20 16:42:11,632 - aiogram.dispatcher - WARNING - Received SIGINT signal
2025-01-20 16:42:11,643 - aiogram.dispatcher - INFO - Polling stopped for bot @TestOGE8_bot id=5141660175 - 'PokemonStrategy'
2025-01-20 16:42:11,644 - aiogram.dispatcher - INFO - Polling stopped
2025-01-20 16:42:11,897 - apscheduler.scheduler - INFO - Removed job da353f7c2b1c43ec91bb21caa3614bd3


Бот остановлен
